```
A crop of this size requires significant logistics to transport produce, soil, fertilizer, and so on. The Elves are very busy pushing things around in carts on some kind of rudimentary system of tracks they've come up with.

Seeing as how cart-and-track systems don't appear in recorded history for another 1000 years, the Elves seem to be making this up as they go along. They haven't even figured out how to avoid collisions yet.

You map out the tracks (your puzzle input) and see where you can help.

Tracks consist of straight paths (| and -), curves (/ and \), and intersections (+). Curves connect exactly two perpendicular pieces of track; for example, this is a closed loop:

/----\
|    |
|    |
\----/
Intersections occur when two perpendicular paths cross. At an intersection, a cart is capable of turning left, turning right, or continuing straight. Here are two loops connected by two intersections:

/-----\
|     |
|  /--+--\
|  |  |  |
\--+--/  |
   |     |
   \-----/
Several carts are also on the tracks. Carts always face either up (^), down (v), left (<), or right (>). (On your initial map, the track under each cart is a straight path matching the direction the cart is facing.)

Each time a cart has the option to turn (by arriving at any intersection), it turns left the first time, goes straight the second time, turns right the third time, and then repeats those directions starting again with left the fourth time, straight the fifth time, and so on. This process is independent of the particular intersection at which the cart has arrived - that is, the cart has no per-intersection memory.

Carts all move at the same speed; they take turns moving a single step at a time. They do this based on their current location: carts on the top row move first (acting from left to right), then carts on the second row move (again from left to right), then carts on the third row, and so on. Once each cart has moved one step, the process repeats; each of these loops is called a tick.

For example, suppose there are two carts on a straight track:

|  |  |  |  |
v  |  |  |  |
|  v  v  |  |
|  |  |  v  X
|  |  ^  ^  |
^  ^  |  |  |
|  |  |  |  |
First, the top cart moves. It is facing down (v), so it moves down one square. Second, the bottom cart moves. It is facing up (^), so it moves up one square. Because all carts have moved, the first tick ends. Then, the process repeats, starting with the first cart. The first cart moves down, then the second cart moves up - right into the first cart, colliding with it! (The location of the crash is marked with an X.) This ends the second and last tick.

Here is a longer example:

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   

/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   

/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   

/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   

/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /--<-\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /<---\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-<--/
  \------/   

/---\        
|   |  v----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \<+--/
  \------/   

/---\        
|   |  /----\
| /-+--v-\  |
| | |  | |  |
\-+-/  ^-+--/
  \------/   

/---\        
|   |  /----\
| /-+--+-\  |
| | |  X |  |
\-+-/  \-+--/
  \------/   
After following their respective paths for a while, the carts eventually crash. To help prevent crashes, you'd like to know the location of the first crash. Locations are given in X,Y coordinates, where the furthest left column is X=0 and the furthest top row is Y=0:

           111
 0123456789012
0/---\        
1|   |  /----\
2| /-+--+-\  |
3| | |  X |  |
4\-+-/  \-+--/
5  \------/   
In this example, the location of the first crash is 7,3.
```

In [1]:
DECISION = [
    x->(x[1], -x[0]), # Turn left
    x->x,             # Straight
    x->(-x[1], x[0])  # Turn right
]
DIRECTIONS = { '>': (1, 0), '<': (-1, 0), 'v': (0, 1), '^': (0, -1)}
TURN = { "/": x->(-x[1],-x[0]), "\\": x->(x[1], x[0]) }

class Cart:
    
    def __init__(self, tracks, x, y):
        self.name = "c{}_{}".format(x, y)
        self.tracks = tracks
        self.x, self.y = x, y
        self.cdirection = DIRECTIONS[self.tracks[y][x]]
        self.tracks[y][x] = '-' if self.tracks[y][x] in "<>" else "|"
        self.ndecision = 0
        
    def tick(self):
        
        # Move one position
        self.x += self.cdirection[0]
        self.y += self.cdirection[1]
        
        # Prepare next move
        t = self.tracks[self.y][self.x]        
        if t in "/\\":
            self.cdirection = TURN[t](self.cdirection)
        elif t == '+':
            self.cdirection = DECISION[self.ndecision](self.cdirection)
            self.ndecision = (self.ndecision + 1) % 3
            
    def __repr__(self):
        return "{}({},{})".format(self.name, self.x, self.y)
        
    
class Map:
    
    def __init__(self, lines):
        
        # Initialize tracks
        self.tracks = [[c for c in l.rstrip("\n")] for l in lines if len(l.strip())>0]
        
        # Initialize carts
        lx = len(self.tracks[0])
        ly = len(self.tracks)
        self.carts = [
            Cart(self.tracks, x, y) 
            for x in range(lx) for y in range(ly)
            if self.tracks[y][x] in "<>v^"
        ]
        
    def tick(self):
        for c in sorted(self.carts, key=x->(x.y, x.x)):
            c.tick()
            
    def crash(self, remove=False):
        positions = {}
        crashed_carts = set()
        for c in self.carts:
            if (c.x, c.y) in positions:
                crashed_carts.add(c)
                crashed_carts.add(positions[(c.x, c.y)])
            positions[(c.x, c.y)] = c
            
        if remove:
            self.carts = [c for c in self.carts if c not in crashed_carts]
            
        return crashed_carts
    
    def __str__(self):
        
        lx = len(self.tracks[0])
        ly = len(self.tracks)
        
        idirection = { v: k for k, v in DIRECTIONS.items()}
        dcarts = {(c.x, c.y): c for c in self.carts}
        
        ptracks = []
        for y in range(ly):
            ptracks.append([])
            for x in range(lx):
                if (x, y) in dcarts:
                    ptracks[y].append(idirection[dcarts[(x, y)].cdirection])
                else:
                    ptracks[y].append(self.tracks[y][x])
                    
        return "\n".join(["".join(t) for t in ptracks] + [""])

In [2]:
with open('input/day13example.txt') as lines:
    field = Map(lines)
    crash = field.crash()
    while len(crash) == 0:
        print(field)
        field.tick()
        crash = field.crash()        
    
crash

/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/-->\        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \->--/
  \------/   

/---v        
|   |  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+>-/
  \------/   

/---\        
|   v  /----\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+->/
  \------/   

/---\        
|   |  /----\
| /->--+-\  |
| | |  | |  |
\-+-/  \-+--^
  \------/   

/---\        
|   |  /----\
| /-+>-+-\  |
| | |  | |  ^
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----\
| /-+->+-\  ^
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /----<
| /-+-->-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /---<\
| /-+--+>\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /--<-\
| /-+--+-v  |
| | |  | |  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /-<--\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   

/---\        
|   |  /<---\
| /-+--+-\  |
| | |  | |  |
\-+-/  \-

{c2_0(7,3), c9_3(7,3)}

In [3]:
with open('input/day13.txt') as lines:
    field = Map(lines)
    crash = field.crash()
    while len(crash) == 0:
        field.tick()
        crash = field.crash()        
    
crash

{c45_24(119,41), c78_19(119,41)}

```
--- Part Two ---
There isn't much you can do to prevent crashes in this ridiculous system. However, by predicting the crashes, the Elves know where to be in advance and instantly remove the two crashing carts the moment any crash occurs.

They can proceed like this for a while, but eventually, they're going to run out of carts. It could be useful to figure out where the last cart that hasn't crashed will end up.

For example:

/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/

/---\  
|   |  
| v-+-\
| | | |
\-+-/ |
  |   |
  ^---^

/---\  
|   |  
| /-+-\
| v | |
\-+-/ |
  ^   ^
  \---/

/---\  
|   |  
| /-+-\
| | | |
\-+-/ ^
  |   |
  \---/
After four very expensive crashes, a tick ends with only one cart remaining; its final location is 6,4.

What is the location of the last cart at the end of the first tick where it is the only cart left?

```

In [4]:
with open('input/day13.txt') as lines:
    field = Map(lines)
    while len(field.carts) > 1:
        field.tick()
        field.crash(remove=True)        
    
field.carts

[c95_41(45,136)]